## Create Langfuse Client

In [1]:
import os

os.environ["LANGFUSE_SECRET_KEY"] = "sk-lf-a92829d1-413f-4994-af78-f3a336efce8a"
os.environ["LANGFUSE_PUBLIC_KEY"] = "pk-lf-60f090e3-f692-4515-8625-cd4b417e9d71"
os.environ["LANGFUSE_HOST"] = "http://localhost:3000"

from langfuse import get_client
 
langfuse = get_client()
 
# Verify connection
if langfuse.auth_check():
    print("Langfuse client is authenticated and ready!")
else:
    print("Authentication failed. Please check your credentials and host.")

Langfuse client is authenticated and ready!


## Create Dataset

In [2]:
import pandas as pd

benchmark_dataset = pd.read_excel("data/sample-question-for-chatbot-new-tanpa-dtt.xlsx")
benchmark_dataset.head()

,NO,ROOT FOLDER,TIPE PERIZINAN,INSTALASI,QUESTION,Expected Respone,Dokumen,Halaman,Unnamed: 8
0,1,PPN PERMIT,PLO,Kombinasi,Sertifikat PLO mana saja pada Subholding PPN y...,"FT Serui,DPPU Kalimarau, TLPG Wayame dll",DPPU Kalimarau,Halaman 2,NaN
1,2,PPN PERMIT,PLO,DPPU Ahmad Yani,Apa lembaga inspeksi yang melakukan inspeksi u...,PT Sucofindo,DPPU Ahmad Yani,Halaman 6,NaN
2,3,PPN PERMIT,PLO,FT Cepu,Berapa sisa umur layan untuk FT Cepu berdasark...,13 Tahun sejak Oktober 2023,6. PLO FT Cepu,Halaman 1,NaN
3,4,PPN PERMIT,PLO,Pipa Penyalur CY,Apa saja instalasi yang dihubungkan oleh pipa ...,"FT Lomanis, FT Maos, FT Rewulu,TBBM Teras",12. PLO Pipa Penyalur CY I dan 13. PLO Pipa P...,CY I Halaman 4 dan CY II Halaman 3,NaN
4,5,PPN PERMIT,PLO,Pipeline perak -Juanda,Apa saja temuan yang di lampirkan oleh pihak i...,a. Memastikan dokumen Penelaahan desain pada s...,Pipeline Perak - Juanda,Halaman 10 - 17,NaN


In [3]:
cleaned_benchmark_dataset = benchmark_dataset.dropna(subset=['Expected Respone']).reset_index(drop=True)
cleaned_benchmark_dataset.tail()

,NO,ROOT FOLDER,TIPE PERIZINAN,INSTALASI,QUESTION,Expected Respone,Dokumen,Halaman,Unnamed: 8
75,76,SHU PERMIT,IZIN LINGKUNGAN,EP Asset 1- Field Pangkalan Susu,Berapa jumlah sumur produksi dan injeksi yang ...,84 Sumur produksi dan 38 sumur Injeksi,3. Izin Lingkungan SKKL No. 361_MenLHK_SETJEN_...,Halaman 8,NaN
76,77,SHU PERMIT,IZIN LINGKUNGAN,EP Asset 1 - Field Jambi,Kapan terakhir persetujuan lingkungan di EP As...,2024,29. SK No 1103 Tahun 2024 Adendum andal RKL RP...,Halaman 1,NaN
77,78,SHU PERMIT,IZIN LINGKUNGAN,EP Asset 1 - Field Rantau,Apa saja Limbah B3 dan Sumber Limbah B3 yang t...,Aki / Baterai Bekas\nSumber: Fasilitas Produks...,2. Izin Lingkungan SKKL No. 361_MenLHK_SETJEN_...,Halaman 69 - 74,NaN
78,79,SHU PERMIT,IZIN LINGKUNGAN,PHE Jambi Merang,Apa saja sumber dampak penting pada tahap kont...,Sumber Dampak : Penerimaan dan Mobilisasi tena...,8. SK.391_Menlhk_Setjen_PLA.4_9_2018 - Izin Li...,Halaman 5 -19,NaN
79,80,SHU PERMIT,IZIN LINGKUNGAN,PHE NSO,Sebutkan ruang lingkup kegiatan yang tertuang ...,\n\n1. Operasional tenaga kerja eksisting.\n\n...,SKKL - SK.319_MENLH_SETJEN_PLA.4_4_2023,Halaman 4-5,NaN


In [ ]:
# DATASET_NAME = "pdh-permit-agent-tanpa-dtt-v2"

# langfuse.create_dataset(
#     name=DATASET_NAME,
#     description="Experiment dataset for PDH Permit Agent dengan tambahan pertanyaan tentang metadata dokumen"
# )

Dataset(id='cmhwyw10y0001nu07yqgwb7c5', name='pdh-permit-agent-tanpa-dtt-v2', description='Experiment dataset for PDH Permit Agent dengan tambahan pertanyaan tentang metadata dokumen', metadata=None, project_id='cmgd98aqj0006on077tnv663m', created_at=datetime.datetime(2025, 11, 13, 5, 7, 8, 722000, tzinfo=datetime.timezone.utc), updated_at=datetime.datetime(2025, 11, 13, 5, 7, 8, 722000, tzinfo=datetime.timezone.utc))

In [ ]:
# for idx, row in cleaned_benchmark_dataset.iterrows():
#     additional_metadata = {
#         "root_folder": row["ROOT FOLDER"],
#         "tipe_perizinan": row["TIPE PERIZINAN"],
#         "instalasi": row["INSTALASI"],
#         "dokumen": row["Dokumen "],
#         "tipe": "Pertanyaan tanpa dtt"
#     }
    
#     langfuse.create_dataset_item(
#         dataset_name=DATASET_NAME,
#         input=row['QUESTION'],
#         expected_output=row['Expected Answer'],
#         metadata=additional_metadata
#     )

## Prepared RAGAS Benchmark

In [4]:
from langchain_openai import AzureChatOpenAI
from ragas.llms.base import LangchainLLMWrapper

evaluator_llm = LangchainLLMWrapper(AzureChatOpenAI(
    openai_api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
    model=os.getenv("AZURE_OPENAI_MODEL_NAME"),
    validate_base_url=False,
))

/Users/ilhamfadhil/Documents/Work/beesar/pdh-permit-agent/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from src.permit_agent.agent_langchain import agent

from ragas.dataset_schema import SingleTurnSample
from ragas.metrics import SimpleCriteriaScore

from langfuse.langchain import CallbackHandler

from pprint import pprint

langfuse_handler = CallbackHandler()

index_docs = 5
user_input = cleaned_benchmark_dataset.iloc[index_docs]['QUESTION']
reference = cleaned_benchmark_dataset.iloc[index_docs]['Expected Respone']
print(f"\n ======= User Input ========\nQuestion: {user_input}\n")
agent_response = await agent.ainvoke({"messages": [user_input]}, config={"recursion_limit": 10, "callbacks": [langfuse_handler]})

sample = SingleTurnSample(
    user_input=user_input,
    response=agent_response["messages"][-1].content,
    reference=reference
)

print("===== Agent Response =======")
pprint(sample.model_dump())

scorer =  SimpleCriteriaScore(
    name="course_grained_score",
    definition="Score 1 to 100 by similarity",
    llm=evaluator_llm
)

score_result = await scorer.single_turn_ascore(sample)
print("\n===== Score Result =======")
print(f"Score result: {score_result}")

Langfuse client is authenticated and ready!

 ======= User Input ========
Question: Sebutkan nomor PLO untuk IT Makassar LPG dan Fuel !

===== Agent Response =======
{'multi_responses': None,
 'reference': '878/49-7/PLO/DMT/2023 dan 879/49-7/PLO/DMT/2023',
 'reference_context_ids': None,
 'reference_contexts': None,
 'response': 'Berikut adalah nomor Persetujuan Layak Operasi (PLO) untuk '
             'Integrated Terminal (IT) Makassar:\n'
             '\n'
             '- **PLO IT Makassar LPG:**  \n'
             '  Nomor: 879/49-7/PLO/DMT/2023  \n'
             '  Tanggal: 8 Desember 2023  \n'
             '  [PLO Integrated Terminal Makassar (LPG), Page 1]\n'
             '\n'
             '- **PLO IT Makassar Fuel (BBM):**  \n'
             '  Nomor: 878/49-7/PLO/DMT/2023  \n'
             '  Tanggal: 8 Desember 2023  \n'
             '  [PLO Integrated Terminal Makassar (Fuel), Page 1]\n'
             '\n'
             'Jadi, nomor PLO untuk IT Makassar LPG adalah '
            

In [6]:
agent_response

{'messages': [HumanMessage(content='Sebutkan nomor PLO untuk IT Makassar LPG dan Fuel !', additional_kwargs={}, response_metadata={}, id='6a3f5285-0523-4623-a645-3ecacab24720'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 65, 'prompt_tokens': 809, 'total_tokens': 874, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_f99638a8d7', 'id': 'chatcmpl-CgyX7Xwvquu6A6AXY2Cwvsp4ItOwZ', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtere

## Running Experiment

In [7]:
async def run_agent(user_input, langfuse_handler):

    agent_response = await agent.ainvoke({"messages": [user_input]}, config={"recursion_limit": 10, "callbacks": [langfuse_handler]})

    return agent_response

In [17]:
await run_agent(user_input, langfuse_handler)

{'messages': [HumanMessage(content='Sebutkan nomor PLO untuk IT Makassar LPG dan Fuel !', additional_kwargs={}, response_metadata={}, id='3591e815-2c11-49fb-82cc-41c4d323cfce'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 65, 'prompt_tokens': 809, 'total_tokens': 874, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_f99638a8d7', 'id': 'chatcmpl-CgxsCjodKsaVKAIFL36hcbmr3xcGJ', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtere

In [8]:
scorer =  SimpleCriteriaScore(
    name="course_grained_score",
    definition="Score 1 to 100 by similarity. Give higher mark if the content seemed correct",
    llm=evaluator_llm
)

async def course_grained_score(user_input: str, response: list, reference: str, scorer: SimpleCriteriaScore = scorer) -> int:
    """
    Score the response on a scale of 1 to 100 based on its similarity to the reference.
    """
    
    sample = SingleTurnSample(
        user_input=user_input,
        response=response["messages"][-1].content,
        reference=reference
        )

    score_result = await scorer.single_turn_ascore(sample)

    return score_result

In [9]:
score_result  = await course_grained_score(user_input, agent_response, reference)
print(f"Score result: {score_result}")

Score result: 100


In [30]:
DATASET_NAME = "pdh-permit-agent-tanpa-dtt-v2"


dataset = get_client().get_dataset(name=DATASET_NAME)

for index, item in enumerate(dataset.items):
    print(f"\n===== Evaluating Item ID: {index} : {item.id} =======\nInput: {item.input}\nExpected Output: {item.expected_output}\n")


===== Evaluating Item ID: 0 : 2a82692a-373e-4ccc-890d-abc264e886ea =======
Input: Sebutkan nomor PLO yang sudah kadaluwarsa di SH KPI  !
Expected Output: 1. 79/48-4/PLO/DMT/2022
2. 533/48-4/PLO/DMT/2021
3. 265/48-5/PLO/DMT/2021
4. 86/48-5/SKPI/18.01/DJM.T/2018
5. 87/48-5/SKPI/18.01/DJM.T/2018
6. 88/48-5/SKPI/18.01/DJM.T/2018
7. 118/48-6/PLO/DMT/2022
8. 648/48-7/PLO/DMT/2022


===== Evaluating Item ID: 1 : 6b82d98c-c1f9-4b99-8a90-6a75a41cab76 =======
Input: Sebutkan Instalasi mana saja yang PLO nya sudah kadaluwarsa di SH PPN !
Expected Output: 1. FT Serui, 2.FT Cikampek


===== Evaluating Item ID: 2 : f09b904c-5feb-472e-a70e-a8af95b9e0c3 =======
Input: Sebutkan Instalasi milik PGN yang memiliki KKPR dengan tanggal terbit paling baru !
Expected Output: KKPR Jargas Kota Bandar Lampung, di terbitkan pada 6 Agustus 2024


===== Evaluating Item ID: 3 : dd22dd8c-238c-43a5-accc-03e60218125f =======
Input: Sebutkan Instalasi milik PGN yang memiliki KKPR dengan tanggal terbit paling lama !
Exp

In [10]:
from langfuse.langchain import CallbackHandler

langfuse_handler = CallbackHandler()

DATASET_NAME = "pdh-permit-agent-tanpa-dtt-v2"
EXPERIMENT_NAME = "Metadata + AI Search - 20251129 gpt-4.1"
DESCRIPTION = "Experiment with semantic ranking search using Azure OpenAI and LangChain agent"
experiment_metadata = {
    "model": os.getenv("AZURE_OPENAI_MODEL_NAME"),
    "temperature": 0.0,
    "max_tokens": 1000,
    "timeout": 500,
    "search_type": "semantic_ranking",
    "semantic_configuration": "content-score",
    "agent_type": "langchain"
}

dataset = get_client().get_dataset(name=DATASET_NAME)

for item in dataset.items:

    with item.run(
        run_name=EXPERIMENT_NAME,
        run_description=DESCRIPTION,
        run_metadata=experiment_metadata
    ) as root_span:

        output = await run_agent(item.input, langfuse_handler)

        root_span.update_trace(input=item.input, output=output['messages'][-1].content)

        root_span.score_trace(
            name="course_grained_score",
            comment="Score 1 to 100 by similarity",
            value=await course_grained_score(item.input, output, str(item.expected_output))
        )

get_client().flush()